## Generation and Transmission Expansion Planning (GTEP) Model

### Sets:
- N: Set of nodes, N = {1, 2, 3}
- E: Set of energy sources, E = {Wind, Hydro}

### Parameters:
- $C_e$: Cost of energy production from source e in $/MWh
- $L_{ij}$: Maximum transmission capacity between node i and node j in MW
- $D_i$: Demand at node i in MWh
- $P_e$: Maximum production capacity of energy source e in MW

### Variables:
- $x_{ie}$: Energy produced from source e at node i in MWh
- $f_{ij}$: Power flow from node i to node j in MW

### Objective Function:
Minimize the total cost of energy production:
min sum(i in N) sum(e in E) C_e * x_ie

### Constraints:
1. **Energy Balance:** \
$\sum_{e}^{E} x_{ie} + \sum_{j \neq i}^{N} f_{ij} - \sum_{j \neq i}^{N} f_{ji} = D_i \; \; \forall \; i \in N$



2. **Transmission Capacity:** \
$-L_{ij} \leq f_{ij} \leq L_{ij} \; \; \forall \; \; (i, j) \in \; \; N, i \neq j$



3. **Production Capacity:** \
$0 <= x_{ie} <= P_e \forall (i, e) \in (N, E)$



In [2]:
from pyomo.environ import *

# Define the model
model = ConcreteModel()

# Define sets
model.N = Set(initialize=[1, 2, 3])  # Nodes
model.E = Set(initialize=['Wind', 'Hydro'])  # Energy sources

# Define parameters with dummy values to avoid the error
model.C_e = Param(model.E, initialize={'Wind': 50, 'Hydro': 40}, within=PositiveReals)  # Cost of energy production
model.L = Param(model.N, model.N, initialize={(1,2): 100, (1,3): 100, (2,1): 100, (2,3): 100, (3,1): 100, (3,2): 100}, default=0, within=NonNegativeReals)  # Transmission capacity
model.D = Param(model.N, initialize={1: 150, 2: 100, 3: 120}, within=NonNegativeReals)  # Demand at nodes
model.P_e = Param(model.E, initialize={'Wind': 200, 'Hydro': 150}, within=NonNegativeReals)  # Production capacity

# Define variables
model.x = Var(model.N, model.E, within=NonNegativeReals)  # Energy produced
model.f = Var(model.N, model.N, within=Reals)  # Power flow

# Define objective function
def obj_rule(m):
    return sum(m.C_e[e] * m.x[i, e] for i in m.N for e in m.E)

model.obj = Objective(rule=obj_rule, sense=minimize)

# Define constraints
def energy_balance_rule(m, i):
    return (sum(m.x[i, e] for e in m.E) +
            sum(m.f[i, j] for j in m.N if i != j) -
            sum(m.f[j, i] for j in m.N if i != j) == m.D[i])

model.energy_balance = Constraint(model.N, rule=energy_balance_rule)

def transmission_capacity_rule(m, i, j):
    if i == j:
        return Constraint.Skip
    return inequality(-m.L[i, j], m.f[i, j], m.L[i, j])

model.transmission_capacity = Constraint(model.N, model.N, rule=transmission_capacity_rule)

def production_capacity_rule(m, i, e):
    return m.x[i, e] <= m.P_e[e]

model.production_capacity = Constraint(model.N, model.E, rule=production_capacity_rule)

# Display the model
model.pprint()


7 Set Declarations
    E : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'Wind', 'Hydro'}
    L_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    N*N :    9 : {(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)}
    N : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    f_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    N*N :    9 : {(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)}
    production_capacity_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    N*E :    6 : {(1, 'Wind'), (1, 'Hydro'), (2, 'Wind'), (2, 'Hydro'), (3, 'Wind'), (3, 'Hydro')}
    transmission_capacity_index : Size=1, In

In [3]:
from pyomo.opt import SolverFactory

# Create a solver
solver = SolverFactory('glpk')

# Solve the model
solver.solve(model, tee=True)

# Display the results
results = {}
for i in model.N:
    for e in model.E:
        results[(i, e)] = model.x[i, e].value
results

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\SAM-WORK\AppData\Local\Temp\tmp_p0xh3oe.glpk.raw --wglp
 C:\Users\SAM-WORK\AppData\Local\Temp\tmpexvx0h0o.glpk.glp --cpxlp C:\Users\SAM-WORK\AppData\Local\Temp\tmp6dsykxev.pyomo.lp
Reading problem data from 'C:\Users\SAM-WORK\AppData\Local\Temp\tmp6dsykxev.pyomo.lp'...
21 rows, 12 columns, 36 non-zeros
126 lines were read
Writing problem data to 'C:\Users\SAM-WORK\AppData\Local\Temp\tmpexvx0h0o.glpk.glp'...
105 lines were written
GLPK Simplex Optimizer 5.0
21 rows, 12 columns, 36 non-zeros
Preprocessing...
3 rows, 9 columns, 15 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 3
*     0: obj =   1.850000000e+04 inf =   0.000e+00 (3)
*     4: obj =   1.480000000e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40432 byt

{(1, 'Wind'): 0.0,
 (1, 'Hydro'): 150.0,
 (2, 'Wind'): 0.0,
 (2, 'Hydro'): 100.0,
 (3, 'Wind'): 0.0,
 (3, 'Hydro'): 120.0}

# Multi-time steps 


In [4]:
from pyomo.environ import *

# Define the model
functional_model = ConcreteModel()

# Sets
functional_model.N = Set(initialize=['Node1', 'Node2', 'Node3'])
functional_model.E = Set(initialize=['Wind', 'Hydro'])
functional_model.T = Set(initialize=range(1, 25))

# Parameters
functional_model.C_e = Param(functional_model.E, functional_model.T, initialize=10, within=PositiveReals)  # Cost of energy production
functional_model.L_ij = Param(functional_model.N, functional_model.N, initialize=100, within=PositiveReals, mutable=True)  # Initial transmission capacity
functional_model.D_it = Param(functional_model.N, functional_model.T, initialize=50, within=PositiveReals, mutable=True)  # Demand
functional_model.P_et = Param(functional_model.E, functional_model.T, initialize=100, within=PositiveReals, mutable=True)  # Initial production capacity
functional_model.C_cap_e = Param(functional_model.E, initialize=1000, within=PositiveReals)  # Cost of building new capacity
functional_model.C_trans = Param(initialize=500, within=PositiveReals)  # Cost of building new transmission lines

# Adjustments to ensure positive flows
# Increase demand in Node1 and Node3
functional_model.D_it['Node1', :] = 80
functional_model.D_it['Node3', :] = 80

# Limit initial generation capacity in Node1 and Node3
functional_model.P_et['Wind', :] = 10
functional_model.P_et['Hydro', :] = 10

In [5]:
# Variables
functional_model.x_iet = Var(functional_model.N, functional_model.E, functional_model.T, within=NonNegativeReals)  # Energy produced
functional_model.f_ijt = Var(functional_model.N, functional_model.N, functional_model.T, within=NonNegativeReals)  # Power flow
functional_model.cap_e = Var(functional_model.E, within=NonNegativeReals)  # New capacity built
functional_model.trans_ij = Var(functional_model.N, functional_model.N, within=NonNegativeReals)  # New transmission lines built

# Objective function
def functional_obj_rule(m):
    return sum(m.C_e[e, t] * m.x_iet[n, e, t] for n in m.N for e in m.E for t in m.T) + sum(m.C_cap_e[e] * m.cap_e[e] for e in m.E) + sum(m.C_trans * m.trans_ij[i, j] for i in m.N for j in m.N)
functional_model.obj = Objective(rule=functional_obj_rule, sense=minimize)

# Constraints
# Energy balance constraint
def functional_energy_balance_rule(m, n, t):
    return sum(m.x_iet[n, e, t] for e in m.E) + sum(m.f_ijt[n, j, t] for j in m.N) - sum(m.f_ijt[i, n, t] for i in m.N) == m.D_it[n, t]
functional_model.energy_balance_constraint = Constraint(functional_model.N, functional_model.T, rule=functional_energy_balance_rule)

# Transmission capacity constraint
def functional_transmission_capacity_rule(m, i, j, t):
    return m.f_ijt[i, j, t] <= m.L_ij[i, j] + m.trans_ij[i, j]
functional_model.transmission_capacity_constraint = Constraint(functional_model.N, functional_model.N, functional_model.T, rule=functional_transmission_capacity_rule)

# Production capacity constraint
def functional_production_capacity_rule(m, n, e, t):
    return m.x_iet[n, e, t] <= m.P_et[e, t] + m.cap_e[e]
functional_model.production_capacity_constraint = Constraint(functional_model.N, functional_model.E, functional_model.T, rule=functional_production_capacity_rule)

In [6]:
functional_model.pprint()

13 Set Declarations
    C_e_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    E*T :   48 : {('Wind', 1), ('Wind', 2), ('Wind', 3), ('Wind', 4), ('Wind', 5), ('Wind', 6), ('Wind', 7), ('Wind', 8), ('Wind', 9), ('Wind', 10), ('Wind', 11), ('Wind', 12), ('Wind', 13), ('Wind', 14), ('Wind', 15), ('Wind', 16), ('Wind', 17), ('Wind', 18), ('Wind', 19), ('Wind', 20), ('Wind', 21), ('Wind', 22), ('Wind', 23), ('Wind', 24), ('Hydro', 1), ('Hydro', 2), ('Hydro', 3), ('Hydro', 4), ('Hydro', 5), ('Hydro', 6), ('Hydro', 7), ('Hydro', 8), ('Hydro', 9), ('Hydro', 10), ('Hydro', 11), ('Hydro', 12), ('Hydro', 13), ('Hydro', 14), ('Hydro', 15), ('Hydro', 16), ('Hydro', 17), ('Hydro', 18), ('Hydro', 19), ('Hydro', 20), ('Hydro', 21), ('Hydro', 22), ('Hydro', 23), ('Hydro', 24)}
    D_it_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    N*T :   72 : {('Node1', 1), ('Node1', 2), (

In [7]:
# Solve the model
solver = SolverFactory('glpk')
solver.solve(functional_model)

# Display the results
results = {}
results['Energy Produced'] = {(n, e, t): functional_model.x_iet[n, e, t].value for n in functional_model.N for e in functional_model.E for t in functional_model.T}
results['Power Flow'] = {(i, j, t): functional_model.f_ijt[i, j, t].value for i in functional_model.N for j in functional_model.N for t in functional_model.T}
results['New Capacity Built'] = {e: functional_model.cap_e[e].value for e in functional_model.E}
results['New Transmission Lines Built'] = {(i, j): functional_model.trans_ij[i, j].value for i in functional_model.N for j in functional_model.N}
results

{'Energy Produced': {('Node1', 'Wind', 1): 10.0,
  ('Node1', 'Wind', 2): 10.0,
  ('Node1', 'Wind', 3): 10.0,
  ('Node1', 'Wind', 4): 10.0,
  ('Node1', 'Wind', 5): 10.0,
  ('Node1', 'Wind', 6): 10.0,
  ('Node1', 'Wind', 7): 10.0,
  ('Node1', 'Wind', 8): 10.0,
  ('Node1', 'Wind', 9): 10.0,
  ('Node1', 'Wind', 10): 10.0,
  ('Node1', 'Wind', 11): 10.0,
  ('Node1', 'Wind', 12): 10.0,
  ('Node1', 'Wind', 13): 10.0,
  ('Node1', 'Wind', 14): 10.0,
  ('Node1', 'Wind', 15): 10.0,
  ('Node1', 'Wind', 16): 10.0,
  ('Node1', 'Wind', 17): 10.0,
  ('Node1', 'Wind', 18): 10.0,
  ('Node1', 'Wind', 19): 10.0,
  ('Node1', 'Wind', 20): 10.0,
  ('Node1', 'Wind', 21): 10.0,
  ('Node1', 'Wind', 22): 10.0,
  ('Node1', 'Wind', 23): 10.0,
  ('Node1', 'Wind', 24): 10.0,
  ('Node1', 'Hydro', 1): 60.0,
  ('Node1', 'Hydro', 2): 60.0,
  ('Node1', 'Hydro', 3): 60.0,
  ('Node1', 'Hydro', 4): 60.0,
  ('Node1', 'Hydro', 5): 60.0,
  ('Node1', 'Hydro', 6): 60.0,
  ('Node1', 'Hydro', 7): 60.0,
  ('Node1', 'Hydro', 8): 60.0